In [1]:
import numpy as np
import pandas as pd
import os 
import glob
from FlowCytometryTools import FCMeasurement
from collections import Counter
import pdb
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import xlrd
from collections import Counter

/Users/joannaf/anaconda3/envs/dl/lib/python3.7/site-packages/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)
/Users/joannaf/anaconda3/envs/dl/lib/python3.7/site-packages/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)


### Preprocessing of Chevrier dataset
This noteboook provides codes for preprocessing of the dataset described in 
https://www.sciencedirect.com/science/article/pii/S0092867417304294?via%3Dihub#mmc2 . 
The files downloaded form the cytobank were passed thorugh chevrier_correct_fcs.R script,
since they were corrupted and it was not possible to load them directly into jupyter notebook.

In [2]:
# set the path!
path = '/Users/joannaf/Desktop/DeepLearning/DL2019/project/data/Dataset5/'

data_dir=path+'data_corrected/'
# code from https://www.mkyong.com/python/python-how-to-list-all-files-in-a-directory/
files = []
for r, d, f in os.walk(data_dir):
    for file in f:
        if '.fcs' in file:
            files.append(os.path.join(r, file))

In [3]:
file_info = dict()
for f in files:
    idx = f.split(data_dir)[-1]
    file_info[idx] = dict()
    file_info[idx]['sample'] = f.split('/')[-1].split('.')[0]
    file_info[idx]['experiment'] = f.split('/')[-2]
    df = FCMeasurement(ID='Sample', datafile=f)
    file_info[idx]['ncells'] = df.data.shape[0]
    file_info[idx]['nchannels'] = df.data.shape[1]
    file_info[idx]['channels'] = [x for x in df.data.columns if 'Di' in x]

/Users/joannaf/anaconda3/envs/dl/lib/python3.7/site-packages/fcsparser/api.py:332: UserWarning: The default channel names (defined by the $PnS parameter in the FCS file) were not unique. To avoid problems in downstream analysis, the channel names have been switched to the alternate channel names defined in the FCS file. To avoid seeing this warning message, explicitly instruct the FCS parser to use the alternate channel names by specifying the channel_naming parameter.
  warnings.warn(msg)


In [27]:
summary_df = pd.DataFrame(index=file_info[list(file_info.keys())[0]].keys(),
                           columns=file_info.keys())

for f in file_info.keys():
    summary_df.loc[:,f] = pd.DataFrame.from_dict(file_info[f], orient='index')
summary_df = summary_df.transpose()
summary_df.head(2)

,sample,experiment,ncells,nchannels,channels
experiment_102007_files/rcc7.fcs,rcc7,experiment_102007_files,11842,55,"[beadDist, Y89Di, Pd102Di, Pd104Di, Pd105Di, P..."
experiment_102007_files/rcc17.fcs,rcc17,experiment_102007_files,14,55,"[beadDist, Y89Di, Pd102Di, Pd104Di, Pd105Di, P..."


In [5]:
# ### load exemplary data
# path_ex = data_dir+'experiment_102007_files/rcc7.fcs'
# df = FCMeasurement(ID='Sample', datafile=path_ex)
# display(df.shape)
# df.data.columns

In [28]:
### clinical info
fname = 'clinical_info.xlsx'
xl_file = pd.ExcelFile(path+fname)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
df_clinical = pd.read_excel(path+fname, sheet_name='Sheet3')
df_batch = pd.read_excel(path+fname, sheet_name='Sheet1')
df_clinical = df_clinical.merge(df_batch, right_on='Sample ID', 
                                left_on=df_clinical.columns[0])

# add clinical info to the summary_df
df_clinical['sample'] = ['rcc'+str(x) for x in df_clinical.iloc[:,0]]
summary_df = summary_df.merge(df_clinical, on='sample')
summary_df.head(2)
### excluded samples: 61 (outlier for myeloid compartment); 
# 25, 17 (insufficient immune cell number)

,sample,experiment,ncells,nchannels,channels,Code used in the file name,Tissue Bank ID#,Subtype,Gender,Age,...,Date Last Follow-up,Status (Alive/Dead),Date of Death,Time to death,Length FU (yr),Cause Death,Case Note,Sample ID,Site,Grade_y
0,rcc7,experiment_102007_files,11842,55,"[beadDist, Y89Di, Pd102Di, Pd104Di, Pd105Di, P...",7,62224,ccRCC,M,57,...,2015-09-10,A,NaT,NaN,NaN,NaN,NaN,7,UHN,3
1,rcc7,experiment_101718_files,30365,55,"[Y89Di, Pd102Di, Pd104Di, Pd105Di, Pd106Di, Pd...",7,62224,ccRCC,M,57,...,2015-09-10,A,NaT,NaN,NaN,NaN,NaN,7,UHN,3


In [7]:
### antibodies
fname = 'antibodies.xlsx'
xl_file = pd.ExcelFile(path+fname)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
#display(dfs)
df_anti_tam = pd.read_excel(path+fname, sheet_name='Sheet1')
df_anti_tcell = pd.read_excel(path+fname, sheet_name='Sheet2')

anti_tam = dict(zip(df_anti_tam.loc[:,'Metal Tag'],
                    df_anti_tam.loc[:,'Target']))

anti_tcell = dict(zip(df_anti_tcell.loc[:,'Metal Tag'],
                    df_anti_tcell.loc[:,'Target']))

# CD86, CD20, CD206, CD68, and CD15 were excluded from the analysis to remove markers
# not expressed on T cells and likely to add noise in the cluster generation process.
# CD7 was also excluded from the PhenoGraph clustering since this marker split most clusters
# into CD7+ and CD7- fractions without a clear biological meaning and simultaneously reduced
# the impact of more biologically relevant markers on the clustering

In [19]:
summary_df.groupby(['experiment'])['nchannels'].apply(lambda x: Counter(x))

experiment                 
experiment_101718_files  55    46.0
                         62     NaN
experiment_101725_files  55     NaN
                         62    46.0
experiment_102007_files  55    46.0
                         62     NaN
experiment_102279_files  55     NaN
                         62    46.0
Name: nchannels, dtype: float64

In [40]:
### combine files (rbind) per experiment
experiments = list(set(summary_df['experiment']))

data_series = pd.Series({})
for exp in experiments:
    files_exp = [x for x in files if exp in x]
    exp_data_series = dict()
    for f in files_exp:
        name = f.split('/')[-1].split('.')[0]
        exp_data_series[name] = FCMeasurement(ID='Sample', datafile=f).data
    data_series[exp] = pd.concat(exp_data_series, axis=0)

In [41]:
data_series['experiment_101718_files'].head(2)

Time  Event_length       Y89Di  Pd102Di     Pd104Di    Pd105Di  \
rcc7 0   282.031006          24.0  212.287338      0.0   76.668678   6.793280   
     1  1409.661011          21.0  313.906921      0.0  114.250526  12.475786   

           Pd106Di   Pd108Di   Pd110Di  In113Di  ...  Yb174Di   Lu175Di  \
rcc7 0  128.322739  2.270094  6.068331      0.0  ...      0.0  0.000000   
     1  186.288620  4.739249  2.172937      0.0  ...      0.0  6.206942   

        Yb176Di    Ir191Di     Ir193Di    Pt194Di    Pt195Di   beadDist  \
rcc7 0      0.0  57.076450  154.620804   7.797725   3.809574  69.986053   
     1      0.0  80.074478  146.657959  23.305716  35.812504  69.775345   

        barcode  phenograph  
rcc7 0      4.0         6.0  
     1      4.0        11.0  

[2 rows x 55 columns]

In [ ]:
# TD: 
# - pass channel names through corresponding dictionaries (first remove missing)
# [anti_tam[x] for x in df.columns]
# - arcsinh
# def asinh(x, c=5):
#     x = np.log(x/c + np.sqrt((x/c)**2 +1))
#     return(x)
# - processing steps as described in the manuscript (remove cells&markers, 99th perc, [0,1])

